# O Problema: PNAD-COVID 19

Você foi contratado por um grande hospital para entender o comportamento da população na época da pandemia da COVID-19 e para definir quais indicadores seriam importantes para o planejamento.

O estudo do PNAD COVID-19 foi identificado como uma ótima base para extrair estas respostas, porém não será necessário utilizar todas as informações coletadas pelo IBGE. Basicamente, a análise deve abranger:

- Características clínicas dos sintomas;

- Características da população;

- Características econômicas da sociedade.

Além disso, alguns requisitos para a análise são:

- Utilizar no máximo 20 questionamentos realizados na pesquisa;

- Utilizar 3 meses de dados para construção da solução.

# Leitura e processamentos dos dados

Como fonte das informações, será utilizada a base de microdados do PNAD COVID-19 pois disponibiliza tabelas em formato .csv já parcialmente pré-processadas para análise de dados. A única desvantagem é a necessidade de "traduzir" os códigos das colunas para nomes que sejam compreensíveis, além de que os microdados são separados por mês de pesquisa, portanto será necessário concatenar os .csvs de três meses diferentes.

Para esta análise, as tabelas de microdados dos últimos três meses de pesquisa foram selecionadas, corresponde aos meses de setembro, outubro e novembro de 2020.

Fonte dos dados: https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados

In [2]:
import pandas as pd
import os

# Leitura dos arquivos brutos

In [3]:
data_folder = './raw_data/'
processed_data_folder = './processed_data/'

pnad_files_path = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.startswith('PNAD')]
pnad_files_path

['./raw_data/PNAD_COVID_092020.csv',
 './raw_data/PNAD_COVID_102020.csv',
 './raw_data/PNAD_COVID_112020.csv']

In [4]:
raw_pnad = pd.read_csv(pnad_files_path[0])

In [5]:
print(f'Colunas: {len(raw_pnad.columns)}')
print(f'Linhas: {len(raw_pnad)}')
display(raw_pnad.head())

Colunas: 145
Linhas: 387298


,Ano,UF,CAPITAL,RM_RIDE,V1008,V1012,V1013,V1016,Estrato,UPA,...,F001,F0021,F0022,F002A1,F002A2,F002A3,F002A4,F002A5,F0061,F006
0,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
1,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
2,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
3,2020,11,11.0,NaN,1,4,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0
4,2020,11,11.0,NaN,2,1,9,5,1110011,110015970,...,1,NaN,NaN,1,1,1,2,1,1,1.0


Tem-se então um total de 145 colunas para cada linha do DataFrame. Como especificado no problema, deve-se reduzir este número para algo em torno de 20 colunas.

Para selecionar as colunas mais valiosas, o dicionário dos dados deve ser consultado.

Dicionário dos dados: https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Documentacao

# Seleção de colunas

**Dicionário de dados das colunas mantidas no dataframe**

- Ano: ano de referência

- UF: Unidade da Federação

- V1013: Mês da pesquisa

- V1022: Situação do domicílio

- V1023: Tipo de área

- A002: idade do morador

- A003: sexo

- A004: Cor ou raça

- A005: escolaridade

- C001: Na semana apssada, por pelo menos uma hora, trabalhou ou fez algum bico?

- C007D: Qual é a principal atividade do local ou empresa em que você trabalha?

- B0011: Na semana passada teve febre?

- B0012: Na semana passada teve tosse?

- B00111: Na semana passa teve perda de cheiro ou sabor?

- B002:  Por causa disso, foi a algum estabelecimento de saúde?

- B005: Ao procurar o hospital, teve que ficar internado por um dia ou mais

- B009B: Resultado exame COVID SWAB

- B009D: Resultado exame com furo no dedo

- B009F: Resultado exame com coleta de sangue na veia

- B011: Na semana passada, devido à pandemia do Coronavírus, em que medida o(a) Sr(a) restringiu o contato com as pessoas? 

In [6]:
# Lista de colunas que serão mantifas no DataFrame
keep_cols = ['Ano','UF','V1013','V1022','V1023', 'A002', 'A003','A004', 'A005','C001','C007D','B0011','B0012', 'B00111','B002','B005','B009B', 'B009D','B009F','B011']

In [7]:
# Criação de novo DataFrame com colunas selecionadas
pnad = raw_pnad[keep_cols]

print(f'N colunas: {len(pnad.columns)}')
pnad

N colunas: 20


,Ano,UF,V1013,V1022,V1023,A002,A003,A004,A005,C001,C007D,B0011,B0012,B00111,B002,B005,B009B,B009D,B009F,B011
0,2020,11,9,1,1,36,1,4,5,1.0,6.0,2,2,2,NaN,NaN,NaN,NaN,NaN,2
1,2020,11,9,1,1,30,2,4,7,1.0,20.0,2,2,2,NaN,NaN,NaN,NaN,NaN,2
2,2020,11,9,1,1,13,1,4,2,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,4
3,2020,11,9,1,1,11,1,4,2,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,4
4,2020,11,9,1,1,57,2,1,2,2.0,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387293,2020,53,9,2,1,45,2,4,3,2.0,17.0,1,1,2,1.0,2.0,1.0,NaN,NaN,4
387294,2020,53,9,2,1,22,2,4,7,2.0,6.0,1,1,2,1.0,2.0,1.0,NaN,NaN,4
387295,2020,53,9,2,1,16,2,4,3,2.0,NaN,1,1,2,1.0,2.0,1.0,NaN,NaN,4
387296,2020,53,9,2,1,83,1,1,2,2.0,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,3


# Valores nulos

In [8]:
pnad.isna().sum()

Ano            0
UF             0
V1013          0
V1022          0
V1023          0
A002           0
A003           0
A004           0
A005           0
C001       67979
C007D     243003
B0011          0
B0012          0
B00111         0
B002      370436
B005      385419
B009B     373214
B009D     368560
B009F     375775
B011           0
dtype: int64

Avaliando as característias das colunas com valores nulos, verifica-se que estão relacionadas às questões clínicas da COVID-19 e também de ocupação com trabalho.

Para a coluna C001, valor nulo representa pessoas que não fazem parte da população trabalhadora, como por exemplo crianças. Já a resposta da coluna C007D parte da premissa que o entrevistado possui um trabalho, portanto os valores nulos representam pessoas que não possuem ocupação atualmente.

Já as colunas relacionadas às questões de saúde da população, todas compreendem apenas os entrevistados que tiveram algum sintoma de COVID-19. Portanto, neste caso, os valores nulos representam as pessoas que não tiveram nenhum indício da doença.

Em todos os casos, os valores nulos serão preenchidos com o **valor 0**, para identificar os entrevistados que não precisaram responder as perguntas descritas acima.

In [9]:
# Colunas com valores nulos
nan_columns = list(pnad.isna().sum().loc[pnad.isna().sum() > 0].index)

In [10]:
# Preenchimento de valores nulos com 0
pnad[nan_columns] = pnad[nan_columns].fillna(0).astype(int)

C:\Users\Jackson Simionato\AppData\Local\Temp\ipykernel_11152\973083516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnad[nan_columns] = pnad[nan_columns].fillna(0).astype(int)


In [11]:
display(pnad.head())

,Ano,UF,V1013,V1022,V1023,A002,A003,A004,A005,C001,C007D,B0011,B0012,B00111,B002,B005,B009B,B009D,B009F,B011
0,2020,11,9,1,1,36,1,4,5,1,6,2,2,2,0,0,0,0,0,2
1,2020,11,9,1,1,30,2,4,7,1,20,2,2,2,0,0,0,0,0,2
2,2020,11,9,1,1,13,1,4,2,0,0,2,2,2,0,0,0,0,0,4
3,2020,11,9,1,1,11,1,4,2,0,0,2,2,2,0,0,0,0,0,4
4,2020,11,9,1,1,57,2,1,2,2,0,2,2,2,0,0,0,0,0,2


In [12]:
# Verificação dos tipos de dado nas colunas
print(pnad.dtypes)

Ano       int64
UF        int64
V1013     int64
V1022     int64
V1023     int64
A002      int64
A003      int64
A004      int64
A005      int64
C001      int32
C007D     int32
B0011     int64
B0012     int64
B00111    int64
B002      int32
B005      int32
B009B     int32
B009D     int32
B009F     int32
B011      int64
dtype: object


# Renomeação de colunas

Na interpretação dos dados é importante que as colunas da base tenham nomes que representem bem o significado da informação.

Para facilitar o controle interno da pesquisa, na base de dados original cada pergunta do questionário é representada por um código único que, por outro lado, dificulta a transformação dos dados em informação.

Por isso, como último passo do processamento dos dados, as colunas serão renomeadas e receberâo nomes que ajudem a indicar do que se trata cada pergunta do questionário.

In [13]:
dicionario_renomear_colunas = {'Ano':'ano', 'UF':'uf', 'V1013':'mes', 'V1022':'zona_moradia','V1023':'regiao_moradia','A002':'idade','A003':'sexo','A004':'cor_raca','A005':'escolaridade',
                               'C001':'trabalhou', 'C007D':'atividade_trabalho', 'B0011':'febre', 'B0012':'tosse','B00111':'olfato_paladar','B002':'procurou_medico',
                               'B005':'internacao','B009B':'covid_swab','B009D':'covid_furo_dedo','B009F':'covid_sangue','B011':'restricao_contato'}

print(len(dicionario_renomear_colunas))

20


In [14]:
pnad.rename(columns=dicionario_renomear_colunas, inplace=True)

pnad.head()

C:\Users\Jackson Simionato\AppData\Local\Temp\ipykernel_11152\502968634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnad.rename(columns=dicionario_renomear_colunas, inplace=True)


,ano,uf,mes,zona_moradia,regiao_moradia,idade,sexo,cor_raca,escolaridade,trabalhou,atividade_trabalho,febre,tosse,olfato_paladar,procurou_medico,internacao,covid_swab,covid_furo_dedo,covid_sangue,restricao_contato
0,2020,11,9,1,1,36,1,4,5,1,6,2,2,2,0,0,0,0,0,2
1,2020,11,9,1,1,30,2,4,7,1,20,2,2,2,0,0,0,0,0,2
2,2020,11,9,1,1,13,1,4,2,0,0,2,2,2,0,0,0,0,0,4
3,2020,11,9,1,1,11,1,4,2,0,0,2,2,2,0,0,0,0,0,4
4,2020,11,9,1,1,57,2,1,2,2,0,2,2,2,0,0,0,0,0,2


# Loop para agregação dos 3 meses de análise

In [15]:
pnad_final_df = pd.DataFrame()
pnad_final_path = os.path.join(processed_data_folder, 'pnad_covid_19_09_10_11_2020.csv')

for file in pnad_files_path:
    raw_pnad = pd.read_csv(file)

    # Novo DataFrame com colunas de interesse
    pnad = raw_pnad[keep_cols]
    
    # Número de linhas e colunas após seleção
    print(f'Colunas: {len(pnad.columns)}')
    print(f'Linhas: {len(pnad)}')
    
    # Colunas com valores nulos
    nan_columns = list(pnad.isna().sum().loc[pnad.isna().sum() > 0].index)

    # Prenchimento de valores nulos
    pnad[nan_columns] = pnad[nan_columns].fillna(0).astype(int)

    if(pnad_files_path.index(file) == 0):
        # Cópia do DataFrame no primeiro loop
        pnad_final_df = pnad.copy()
    else:
        # Concatenação de DataFrames nos loops seguintes
        pnad_final_df = pd.concat([pnad_final_df, pnad], axis=0)

# Criação de novo índice
pnad_final_df.reset_index(drop=True, inplace=True)

# Renomeação de colunas
pnad_final_df.rename(columns=dicionario_renomear_colunas, inplace=True)

# Exportação do arquivo
pnad_final_df.to_csv(pnad_final_path, index=False)

Colunas: 20
Linhas: 387298


C:\Users\Jackson Simionato\AppData\Local\Temp\ipykernel_11152\1099683941.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnad[nan_columns] = pnad[nan_columns].fillna(0).astype(int)


Colunas: 20
Linhas: 380461


C:\Users\Jackson Simionato\AppData\Local\Temp\ipykernel_11152\1099683941.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnad[nan_columns] = pnad[nan_columns].fillna(0).astype(int)


Colunas: 20
Linhas: 381438


C:\Users\Jackson Simionato\AppData\Local\Temp\ipykernel_11152\1099683941.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnad[nan_columns] = pnad[nan_columns].fillna(0).astype(int)


In [24]:
pnad_final_df.head()

,ano,uf,mes,zona_moradia,regiao_moradia,idade,sexo,cor_raca,escolaridade,trabalhou,atividade_trabalho,febre,tosse,olfato_paladar,procurou_medico,internacao,covid_swab,covid_furo_dedo,covid_sangue,restricao_contato
0,2020,11,9,1,1,36,1,4,5,1,6,2,2,2,0,0,0,0,0,2
1,2020,11,9,1,1,30,2,4,7,1,20,2,2,2,0,0,0,0,0,2
2,2020,11,9,1,1,13,1,4,2,0,0,2,2,2,0,0,0,0,0,4
3,2020,11,9,1,1,11,1,4,2,0,0,2,2,2,0,0,0,0,0,4
4,2020,11,9,1,1,57,2,1,2,2,0,2,2,2,0,0,0,0,0,2


# Upload para Google Big Query

Por fim, com os dados processados, será realizado upload para o Google Big Query, uma ferramenta que permitirá maior agilidade e desempenho no armazenamento e consulta dados do PNAD COVID-19

In [21]:
# Função responsável pelo upload da tabela
from google.cloud import bigquery

def create_bigquery_table(df, dataset_tablename, gcp_project_name):
    # df: your pandas dataframe
    # dataset_tablename (str.str): dataset_name.tablename
    # gcp_project_name: GCP Project ID
    
    df.to_gbq(
        destination_table=dataset_tablename,
        project_id=gcp_project_name,
        if_exists="replace",  # 3 available methods: fail/replace/append
    )

In [22]:
# Variáveis indicando projeto, dataset e nome da tabela a ser criada
project_id = 'brave-tea-400210'
dataset_id = 'fase_3_tech_challenge'
table_id = 'pnad-covid-19'
location = 'US'

In [ ]:
# Variáveis indicando projeto, dataset e nome da tabela a ser criada
project_id = 'brave-tea-400210'
dataset_id = 'fase_3_tech_challenge'
table_id = 'pnad-covid-19'
location = 'US'

In [23]:
create_bigquery_table(pnad_final_df, f'{dataset_id}.{table_id}', project_id)